In [37]:
import cv2
import numpy as np
from utils import *
from forward import *


In [38]:
image = cv2.imread("D:\\Dat_Hoctap\\2023\\Tamper_Recover\\Data\\Lenna.png",0)
w_image = image_gen_emb_w(image,137)

(array([138], dtype=int64), array([164], dtype=int64))
[1 1 1 0 1 1 1 1 1 1 0 0]
[0 1 1 1 1 1 0 0 0 1 0 1]


In [39]:
ir = 10
half = 128
ic = 164
A = image[ir*2:ir*2 + 2,ic*2 :ic*2 + 2].copy()
B = image[(ir+half)*2:(ir+half)*2 + 2,ic*2:ic*2 + 2].copy()
C = image[0:2,0:2].copy()

In [40]:
print(C)

[[169 169]
 [169 169]]


In [41]:
print(watermark_gen(A,B))
newC = one_block_emb(C,watermark_gen(A,B))
print(newC)
for e in newC:
    for ee in e:
        print(np.unpackbits(ee))

[0 1 1 1 1 1 0 0 0 1 0 1]
[[171 167]
 [168 173]]
[1 0 1 0 1 0 1 1]
[1 0 1 0 0 1 1 1]
[1 0 1 0 1 0 0 0]
[1 0 1 0 1 1 0 1]


In [42]:
bits = np.unpackbits(C[0,1])
nbits = bits.copy()
nbits[5] = 1
nbits[6] = 1
print(nbits)
print(int(np.packbits(nbits))- int(C[0,1]))

[1 0 1 0 1 1 1 1]
6


In [43]:
N = int(512/2)*int(512/2)
print(N)
for i in range(N):
    if (137*(i) % N + 1) % N == 0:
        print(i)

65536
35399


In [44]:
A = np.array([[1,2],[3,4]])
print(np.where(A==1))

(array([0], dtype=int64), array([0], dtype=int64))


In [45]:
def watermark_extract(A):
    joint_12bits = np.zeros(12,dtype=np.uint8)
    i = 0
    for e in A.flatten():
        joint_12bits[i:i+3] = np.unpackbits(e)[-3:]
        i += 3
    return joint_12bits

In [46]:
print(np.pad(A,1,'constant',constant_values=True))

[[1 1 1 1]
 [1 1 2 1]
 [1 3 4 1]
 [1 1 1 1]]


In [55]:
def level_two_check(lv1_table):
    lv1_table_pad = np.pad(lv1_table,1,'constant',constant_values=True)
    lv2_table = lv1_table.copy()
    list_index_valid = np.where(lv1_table==True)
    tribles = [[[0,-1],[-1,-1],[-1,0]],[[-1,0],[-1,1],[0,1]],[[0,1],[1,1],[1,0]],[[1,0],[1,-1],[0,-1]]]
    for i in range(len(list_index_valid[0])):
        ir = list_index_valid[0][i]
        ic = list_index_valid[1][i]
        ipr = ir + 1
        ipc = ic + 1
        for trible in tribles: 
            valid = False
            for dr,dc in trible:
                valid = valid or lv1_table_pad[ipr+dr,ipc+dc]
            if not valid:
                lv2_table[ir,ic] = False
                break
    return lv2_table

In [69]:
def level_three_check(lv2_table):
    lv2_table_pad = np.pad(lv2_table,1,'constant',constant_values=True)
    lv3_table = lv2_table.copy()
    list_index_valid = np.where(lv2_table==True)
    neighbors = [[0,-1],[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1]]
    for i in range(len(list_index_valid[0])):
        ir = list_index_valid[0][i]
        ic = list_index_valid[1][i]
        ipr = ir + 1
        ipc = ic + 1
        count = 0
        for dr,dc in neighbors:
            if not lv2_table_pad[ipr+dr,ipc+dc]:
                count += 1
                if count>=5:
                    lv3_table[ir,ic] = False
                    break
    return lv3_table    

In [68]:
a = np.arange(9).reshape((3,3))
neighbors = [[0,-1],[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1]]
i = 1
j = 1
for dr,dc in neighbors:
    print(a[i+dr,j+dc])
print(a)

3
0
1
2
5
8
7
6
[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [48]:
table = np.ones((3,3),dtype=np.bool_)
table[1,2] = False
table[2,1] = False
table[2,2] = False
# table[0,1] = False
ipr = 1
ipc = 1
tribles = [[[0,-1],[-1,-1],[-1,0]],[[-1,0],[-1,1],[0,1]],[[0,1],[1,1],[1,0]],[[1,0],[1,-1],[0,-1]]]
for trible in tribles:
    print(trible)
    valid = False
    for dr,dc in trible:
        valid = valid or table[ipr+dr,ipc+dc]
    if not valid:
        print("change to invalid")
        break
    

[[0, -1], [-1, -1], [-1, 0]]
[[-1, 0], [-1, 1], [0, 1]]
[[0, 1], [1, 1], [1, 0]]
change to invalid


In [49]:
print(table)

[[ True  True  True]
 [ True  True False]
 [ True False False]]


In [50]:
def detection(w_image,key):
    nr,nc = w_image.shape[:2]
    nr_table = int(nr/2)
    nc_table = int(nc/2)
    lu_table = create_luck_up_table(nr_table,nc_table,key)
    level_one = np.ones((nr_table,nc_table),dtype = np.bool)
    half = int(nr_table/2)
    # Step1 level 1 detection
    for ir in range(nr_table):
        for ic in range(nc_table):
            A = w_image[ir*2:ir*2 + 2,ic*2 :ic*2 + 2].copy()
            # B = w_image[(ir+half)*2:(ir+half)*2 + 2,ic*2:ic*2 + 2].copy()
            # ir_co_A = int(lu_table[ir,ic]/nc_table)
            # ic_co_A = int(lu_table[ir,ic]%nc_table)
            # ir_co_B = int(lu_table[ir+half,ic]/nc_table)
            # ic_co_B = int(lu_table[ir+half,ic]%nc_table)
            joint_12bits = watermark_extract(A)
            lv_one_check = level_one_check(joint_12bits) 
            if lv_one_check == 0 : 
                print("invalid",ir,ic,joint_12bits)
                level_one[ir,ic] = False
    # Step2 Level 2 detection:
    
    return level_one

In [51]:
level_one = detection(w_image=w_image,key=137)

C:\Users\DuyDat\AppData\Local\Temp\ipykernel_10940\260391646.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  level_one = np.ones((nr_table,nc_table),dtype = np.bool)


In [52]:

print(level_one)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [70]:
level_two = level_two_check(level_one)
level_three = level_three_check(level_two)

In [71]:
print(np.where(level_three==False))

(array([], dtype=int64), array([], dtype=int64))
